<a href="https://colab.research.google.com/github/romanzes637/tensor/blob/master/clust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import numpy as np

paths = ['/gdrive/My Drive/Colab Notebooks/tube10El/{}/devStr'.format(x) for x in range(1, 35)]
times = list()
ts= list()
# initial state
# ts.append(np.zeros((n, 6)))
# times.append(np.full(n, 0))
# other steps
m = 0
n = 16000
for i, p in enumerate(paths):
  with open(p) as f:
    lines = f.readlines()
    x = np.array([[float(y) for y in x.strip()[1:-1].split()] for x in lines[18:-5]])[m:m+n]
    ts.append(x)
    times.append(np.full(len(x), i + 1))
ts = np.vstack(ts)
times = np.hstack(times)
print(len(ts), ts[0], ts[-1])
print(len(times), times[0], times[-1])

In [0]:
def tensor_sort(t):
  at_map = {
      (1, 2): 3, (2, 1): 3,
      (0, 2): 4, (2, 0): 4,
      (0, 1): 5, (1, 0): 5
  }
  st = t[:3]
  sort_st = np.argsort(-np.abs(st))
  ind = np.array([sort_st[0], 
                  sort_st[1], 
                  sort_st[2], 
                  at_map[(sort_st[0], sort_st[1])],
                  at_map[(sort_st[1], sort_st[2])],
                  at_map[(sort_st[2], sort_st[0])]])
  new_t = t[ind]
  return new_t
def tensors_stat(ts):
  stat = dict()
  for t in ts:
    tt = tuple(t)
    stat[tt] = stat.get(tt, 0) + 1
  return stat
   
def tensors_groups(ts, ndigits=2):
  groups = dict()
  for i, t in enumerate(ts): 
    tt = tuple(round(x, ndigits) for x in t)
    groups.setdefault(tt, list()).append(i)
  map_tensor_group = dict()
  for gi, (k, v) in enumerate(groups.items()):
    for ti in v:
      map_tensor_group[ti] = gi
  return groups, map_tensor_group

# t = np.array([1, 2, -3, 2, 0, 1])
# tensor_sort(t)
# ts = np.array([
#     [1, 2, -3, 2, 0, 1],
#     [1, 2, -3, 2, 0, 1]
# ])
# print(ts)
# ts = np.apply_along_axis(tensor_sort, 1, ts)
# print(ts)
# ts = np.apply_along_axis(
#     lambda x: x / max(np.abs(x)) if max(np.abs(x)) != 0 else x, 1, ts)
# print(ts)
# tensors_group(ts, 2)

In [0]:
# Sort tensors (max, int, min, max-int, int-min, min-max)
X = ts.copy()
print(len(X))
print(X[-1])
X = np.apply_along_axis(tensor_sort, 1, X)
print(X[-1])
print(ts[-1])
# Normalize to interval [-1, 1]
X = np.apply_along_axis(
    lambda x: x / max(np.abs(x)) if max(np.abs(x)) != 0 else x, 1, X)
print(X[-1])
# Normalize to distinct: -1, 0 and 1, 0.5, -0.5
X2 = X.copy()
# print(X[-1])
# print(X2[-1])
# X2[(X2 > -1e-3) & (X2 < 1e-3)] = 0
# # X2[(X2 > 0) & (X2 <= 0.5)] = 0.5
# # X2[(X2 < 0) & (X2 >= -0.5)] = -0.5
# # X2[(X2 > 0.5)] = 1
# # X2[(X2 < -0.5)] = -1
# X2[X2 > 0] = 1
# X2[X2 < 0] = -1
# print(X[-1])
# print(X2[-1])
# Groups of identical tensors
gs, map_tg = tensors_groups(X, 3)
print(len(gs))
# print(gs)
X3 = np.array(list(gs))
# print(X3)
# Evaluate distinct categories
X3D = X3.copy()
X3D[(X3D > -1e-3) & (X3D < 1e-3)] = 0
# X3D[(X3D > 0) & (X3D <= 0.5)] = 0.5
# X3D[(X3D < 0) & (X3D >= -0.5)] = -0.5
# X3D[(X3D > 0.5)] = 1
# X3D[(X3D < -0.5)] = -1
X3D[X3D > 0] = 1
X3D[X3D < 0] = -1
print(X3D[-1])
x3d_gs, x3d_tg = tensors_groups(X3D)

In [0]:
from time import time

import matplotlib.pyplot as plt
from matplotlib import style
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold, datasets, preprocessing

style.use('default')
# Next line to silence pyflakes. This import is needed.
Axes3D  

fig = plt.figure(figsize=(16, 8))
n_points = len(X3)
n_neighbors = 10
n_components = 2
# plt.suptitle("Manifold Learning with %i points, %i neighbors"
#              % (n_points, n_neighbors), fontsize=14)
# color = np.array([x3d_tg[i] for i, _ in enumerate(X3)])
color = np.ones(len(X3))  # black


# ax = fig.add_subplot(251, projection='3d')
# ax.scatter(X3[:, 0], X3[:, 1], X3[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.view_init(4, -72)

methods = ['standard', 'ltsa', 'hessian', 'modified']
labels = ['LLE', 'LTSA', 'Hessian LLE', 'Modified LLE']

YS = list()
for i, method in enumerate(methods):
    t0 = time()
    Y = manifold.LocallyLinearEmbedding(n_neighbors, n_components,
                                        eigen_solver='auto',
                                        method=method).fit_transform(X3)
    t1 = time()
    print("%s: %.2g sec" % (methods[i], t1 - t0))

    ax = fig.add_subplot(241 + i)
    plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.gray)
#     plt.title("%s (%.2g sec)" % (labels[i], t1 - t0))
    plt.title("%s" % (labels[i]))
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    plt.axis('tight')
    YS.append(Y)

t0 = time()
Y_iso = manifold.Isomap(n_neighbors, n_components).fit_transform(X3)
t1 = time()
print("Isomap: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(245)
plt.scatter(Y_iso[:, 0], Y_iso[:, 1], c=color, cmap=plt.cm.gray)
plt.title("Isomap")
# plt.title("Isomap (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')


t0 = time()
mds = manifold.MDS(n_components, max_iter=100, n_init=1)
Y_mds = mds.fit_transform(X3)
t1 = time()
print("MDS: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(246)
plt.scatter(Y_mds[:, 0], Y_mds[:, 1], c=color, cmap=plt.cm.gray)
plt.title("MDS")
# plt.title("MDS (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')


t0 = time()
se = manifold.SpectralEmbedding(n_components=n_components,
                                n_neighbors=n_neighbors)
Y_se = se.fit_transform(X3)
t1 = time()
print("SpectralEmbedding: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(247)
plt.scatter(Y_se[:, 0], Y_se[:, 1], c=color, cmap=plt.cm.gray)
plt.title("SpectralEmbedding")
# plt.title("SpectralEmbedding (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

t0 = time()
tsne = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
Y_tsne = tsne.fit_transform(X3)
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(248)
plt.scatter(Y_tsne[:, 0], Y_tsne[:, 1], c=color, cmap=plt.cm.gray)
plt.title("t-SNE")
# plt.title("t-SNE (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

plt.show()

In [0]:
from time import time
import pickle

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold, datasets, preprocessing

# Next line to silence pyflakes. This import is needed.
Axes3D  

fig = plt.figure(figsize=(8, 8))
update = True
name = 'test2'
if update:  # rate is about of 10000 points/sec
  n_points = len(X3)
  print(n_points)
  n_neighbors = 10
  n_components = 2

  t0 = time()
  tsne = manifold.TSNE(n_components=n_components, 
                       perplexity=50.,
                       early_exaggeration=12.,
                       learning_rate=200.,
                       init='random',
                       n_iter=1000,
                       verbose=1,
                       method='barnes_hut',
                       random_state=None)
  Y4S = tsne.fit_transform(X3)
  t1 = time()
  print("t-SNE: %.2g sec" % (t1 - t0))
  # plt.title("t-SNE (%.2g sec)" % (t1 - t0))
  np.save('/gdrive/My Drive/Colab Notebooks/Y4S_{}'.format(name), Y4S)
  Y4 = np.array([Y4S[map_tg[i]] for i, _ in enumerate(X)])
  np.save('/gdrive/My Drive/Colab Notebooks/Y4_{}'.format(name), Y4)
  np.save('/gdrive/My Drive/Colab Notebooks/X_{}'.format(name), X)
  np.save('/gdrive/My Drive/Colab Notebooks/X2_{}'.format(name), X2)
  np.save('/gdrive/My Drive/Colab Notebooks/X3_{}'.format(name), X3)
  np.save('/gdrive/My Drive/Colab Notebooks/ts_{}'.format(name), ts)
  np.save('/gdrive/My Drive/Colab Notebooks/times_{}'.format(name), times)
  with open('/gdrive/My Drive/Colab Notebooks/groups_{}.pickle'.format(name), 'wb') as f:
    pickle.dump(gs, f, protocol=pickle.HIGHEST_PROTOCOL)
  with open('/gdrive/My Drive/Colab Notebooks/map_tg_{}.pickle'.format(name), 'wb') as f:
    pickle.dump(map_tg, f, protocol=pickle.HIGHEST_PROTOCOL)
  with open('/gdrive/My Drive/Colab Notebooks/x3d_gs_{}.pickle'.format(name), 'wb') as f:
    pickle.dump(x3d_gs, f, protocol=pickle.HIGHEST_PROTOCOL)
  with open('/gdrive/My Drive/Colab Notebooks/x3d_tg_{}.pickle'.format(name), 'wb') as f:
    pickle.dump(x3d_tg, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
  print('loading')
  Y4 = np.load('/gdrive/My Drive/Colab Notebooks/Y4_{}.npy'.format(name))
  Y4S = np.load('/gdrive/My Drive/Colab Notebooks/Y4S_{}.npy'.format(name))
  X = np.load('/gdrive/My Drive/Colab Notebooks/X_{}.npy'.format(name))
  X2 = np.load('/gdrive/My Drive/Colab Notebooks/X2_{}.npy'.format(name))
  X3 = np.load('/gdrive/My Drive/Colab Notebooks/X3_{}.npy'.format(name))
  ts = np.load('/gdrive/My Drive/Colab Notebooks/ts_{}.npy'.format(name))
  times = np.load('/gdrive/My Drive/Colab Notebooks/times_{}.npy'.format(name))
  with open('/gdrive/My Drive/Colab Notebooks/groups_{}.pickle'.format(name), 'rb') as f:
    gs = pickle.load(f)
  with open('/gdrive/My Drive/Colab Notebooks/map_tg_{}.pickle'.format(name), 'rb') as f:
    map_tg = pickle.load(f)
  with open('/gdrive/My Drive/Colab Notebooks/x3d_gs_{}.pickle'.format(name), 'rb') as f:
    x3d_gs = pickle.load(f)
  with open('/gdrive/My Drive/Colab Notebooks/x3d_tg_{}.pickle'.format(name), 'rb') as f:
    x3d_tg = pickle.load(f)
print(Y4.shape)
# Colors
# print(color[-1])
# color = color / max(color)
# print(color[-1])
scolor = np.array([x3d_tg[i] for i, _ in enumerate(X3)])
# print(scolor[1])
# ax = fig.add_subplot(2, 5, 10)
ax = fig.add_subplot(1, 1, 1)
plt.scatter(Y4S[:, 0], Y4S[:, 1], c=scolor, cmap=plt.cm.tab10)
# plt.scatter(Y4[:, 0], Y4[:, 1], c=color, cmap=plt.cm.viridis)
# start = 30000
# end = 35001
# plt.scatter(Y4[start:end, 0], Y4[start:end, 1], c=color[start:end], cmap=plt.cm.viridis)
# ax.set_facecolor('black')
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.set_aspect('equal')
plt.tight_layout()
plt.axis('off')
plt.show()

In [0]:
%matplotlib inline

from sklearn.cluster import DBSCAN
from itertools import cycle, islice
from ipywidgets import interact, interactive, interact_manual
import matplotlib.pyplot as plt

YC = Y4S
labels = [0 for x in YC]
def clusterize(eps=3., min_samples=2):
  clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(YC)
  global labels 
  labels = clustering.labels_.astype(np.int)
  labels = [x + 1 for x in labels]
#   print(len(y_pred), len(Y_mds))
  print(set(labels))
#   colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
#                                        '#f781bf', '#a65628', '#984ea3',
#                                        '#999999', '#e41a1c', '#dede00']),
#                                         int(max(labels) + 1))))
#   add black color for outliers (if any)
#   colors = np.append(colors, ["#000000"])
  plt.figure(figsize=(5, 5))
#   plt.scatter(YC[:, 0], YC[:, 1], s=10, color=colors[labels])
  plt.scatter(YC[:, 0], YC[:, 1], s=10, c=labels, cmap=plt.cm.tab10)
  ax.set_aspect('equal')
  plt.tight_layout()
  plt.show()
  return labels

p = interactive(clusterize, eps=(0, 100, .05), min_samples=(1, 100, 1))
p

In [0]:
import pandas as pd

groups = [' | '.join(map('{}'.format, x)) for x in x3d_gs]
kind = np.array([groups[x3d_tg[i]] for i, _ in enumerate(X3)])
cnt = np.array([len(x) for x in gs.values()])
clusters_tensors = dict()
for i, label in enumerate(labels):
  clusters_tensors.setdefault(label, list()).append(i)
clusters_means = dict()
for k, v in clusters_tensors.items():
  ts = [X3[x] for x in v]
  m = np.mean(ts, axis=0)
  clusters_means[k] = ' | '.join(map('{: .2f}'.format, m))
clusters = ['{:{align}{width}}. '.format(x, align='>', width='3') + clusters_means[x] for i, x in enumerate(labels)]
tags = ['tag_{}'.format(i) for i, _ in enumerate(Y4S)]
df = pd.DataFrame({'x': Y4S[:, 0], 'y': Y4S[:, 1], 'kind': kind, 'cnt': cnt,
                   'cluster': clusters, 'tag': tags,
                   '11': X3[:, 0], '22': X3[:, 1], '33': X3[:, 2],
                   '12': X3[:, 3], '23': X3[:, 4], '13': X3[:, 5]})

In [0]:
import altair as alt
alt.data_transformers.enable(max_rows=None)
alt.renderers.set_embed_options(theme='Default')  # 'Default'

interval = alt.selection_interval(
  zoom=False,
  on='[mousedown[event.button === 0], window:mouseup] > window:mousemove!'
#   clear='[mousedown[event.button === 2], window:mouseup] > window:mousemove!'
)
scale = alt.selection_interval(
  bind='scales', 
  translate='[mousedown[event.button === 1], window:mouseup] > window:mousemove!')
size = 350

# dropdown = alt.binding_select(options=groups)
# selection = alt.selection_single(fields=['kind'], bind=dropdown, name='kind')

points = alt.Chart(width=size, height=size).transform_calculate(
    url='https://www.google.com/search?q=' + alt.datum.cnt
).mark_circle().encode(
  alt.X('x:Q', axis=alt.Axis(title='', grid=False)),
  alt.Y('y:Q', axis=alt.Axis(title='', grid=False)),
  color=alt.condition(interval, alt.Color(
      'cluster:N', legend=alt.Legend(labelLimit=200), sort=None), alt.value('lightgrey')),
  size='cnt:Q',
  href='url:N',
  tooltip=['cnt:Q', '11:N', '22:N', '33:N', '12:N', '23:N', '13:N', 
           'cluster:N', 'tag:N', 'url:N']
).add_selection(
    scale,
    interval)
#     selection
# ).transform_filter(selection)

bar = alt.Chart().transform_filter(interval).transform_aggregate(
    mn11 = 'mean(11)', md11 = 'median(11)',
    mn22 = 'mean(22)', md22 = 'median(22)',
    mn33 = 'mean(33)', md33 = 'median(33)',
    mn12 = 'mean(12)', md12 = 'median(12)',
    mn23 = 'mean(23)', md23 = 'median(23)',
    mn13 = 'mean(13)', md13 = 'median(13)'
).transform_fold(['mn11', 'md11', 'mn22', 'md22', 'mn33', 'md33',
                  'mn12', 'md12', 'mn23', 'md23', 'mn13', 'md13']).mark_bar(
    stroke='transparent').encode(
    x=alt.X('key:N', axis=alt.Axis(title='component'), 
            sort=['mn11', 'md11', 'mn22', 'md22', 'mn33', 'md33',
                  'mn12', 'md12', 'mn23', 'md23', 'mn13', 'md13']),
    y=alt.Y('value:Q', axis=alt.Axis(title='mean, median'),
            scale=alt.Scale(domain=(-1, 1))),
    tooltip=['mn11:Q', 'md11:Q', 'mn22:Q', 'md22:Q', 'mn33:Q', 'md33:Q',
             'mn12:Q', 'md12:Q', 'mn23:Q', 'md23:Q', 'mn13:Q', 'md13:Q']
#     color=alt.Color('key:N', legend=None, scale=alt.Scale(
#         scheme=alt.SchemeParams(name='dark2', count=2))),
    ).properties(height=100, width=size)

text = alt.Chart().mark_text(dy=-60).encode(text='sum(cnt):Q').transform_filter(interval)

tensors = alt.vconcat(points, bar + text, data=df)

tensors.save('/gdrive/My Drive/Colab Notebooks/tensors.html')
tensors

In [0]:
import matplotlib.pyplot as plt
from matplotlib import animation, style, cm
style.use('dark_background')
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
cmap = cm.get_cmap('viridis')
colors = cmap(color)
def draw_3d_update(frame, m, n):
  d = 3
  i = int(frame / m)
  k = frame / m - i
  prev_start = i * n
  prev_end = i * n + n
  start = (i + 1) * n
  end = (i + 1) * n + n
  if end > len(Y4):
    start = prev_start
    end = prev_end
  print(n, m, i, i + 1, prev_start, prev_end, start, end, k)
  cs = colors[start:end]
  ps = Y4[start:end]
  prev_ps = Y4[prev_start:prev_end]
  prev_cs = colors[prev_start:prev_end]
  delta_ps = ps - prev_ps
  delta_cs = cs - prev_cs
  cur_ps = prev_ps + k * delta_ps
  cur_cs = prev_cs + k * delta_cs
  ax.cla()
  ax.scatter(cur_ps[:, 0], cur_ps[:, 1], s=1**2, c=cur_cs)
  ax.set_xlim(min(Y4[:, 0]) - d, max(Y4[:, 0]) + d)
  ax.set_ylim(min(Y4[:, 1]) - d, max(Y4[:, 1]) + d)
#   ax.set_xlim(-6.5, -2)
#   ax.set_ylim(1, 5.5)
# ax.xaxis.set_major_formatter(NullFormatter())
# ax.yaxis.set_major_formatter(NullFormatter())
  ax.set_aspect('equal')
  plt.axis('off')
  plt.tight_layout()
steps = 35
m = 5
n = 16000
ani = animation.FuncAnimation(fig, draw_3d_update, (steps - 1)*m + 1, 
                              fargs=[m, n], interval=200, blit=False)
ani.save('/gdrive/My Drive/Colab Notebooks/test2.gif', writer='imagemagick')

In [0]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

style.use('default')

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(10*4, 8*4))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .1,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

# datasets = [
#     (noisy_circles, {'damping': .77, 'preference': -240,
#                      'quantile': .2, 'n_clusters': 2,
#                      'min_samples': 20, 'xi': 0.25}),
#     (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
#     (varied, {'eps': .18, 'n_neighbors': 2,
#               'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
#     (aniso, {'eps': .15, 'n_neighbors': 2,
#              'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
#     (blobs, {}),
#     (no_structure, {})]
datasets = [
    ((YS[0], None), {}),
    ((YS[1], None), {}),
    ((YS[2], None), {}),
    ((YS[3], None), {}),
    ((Y_iso, None), {}),
    ((Y_mds, None), {}),
    ((Y_se, None), {}),
    ((Y_tsne, None), {})
]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
#         plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
#                  transform=plt.gca().transAxes, size=15,
#                  horizontalalignment='right')
        plot_num += 1

plt.show()